# Download historical equity data for NASDAQ stocks from yahoo finance

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from time import time
from pathlib import Path
import pandas as pd

from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import yfinance as yf
from concurrent import futures

In [3]:
idx = pd.IndexSlice

In [4]:
results_path = Path('results', 'asset_pricing')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [5]:
def chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [6]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}'

## Get NASDAQ symbols

In [7]:
traded_symbols = get_nasdaq_symbols()

In [8]:
traded_symbols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10605 entries, A to ZYXI
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Nasdaq Traded     10605 non-null  bool    
 1   Security Name     10605 non-null  object  
 2   Listing Exchange  10605 non-null  category
 3   Market Category   10605 non-null  object  
 4   ETF               10605 non-null  bool    
 5   Round Lot Size    10605 non-null  float64 
 6   Test Issue        10605 non-null  bool    
 7   Financial Status  4633 non-null   category
 8   CQS Symbol        5972 non-null   object  
 9   NASDAQ Symbol     10605 non-null  object  
 10  NextShares        10605 non-null  bool    
dtypes: bool(4), category(2), float64(1), object(4)
memory usage: 559.7+ KB


In [9]:
traded_symbols

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAA,True,Listed Funds Trust AAF First Priority CLO Bond...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,P,,True,100.0,False,NaN,AAAU,AAAU,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False
...,...,...,...,...,...,...,...,...,...,...,...
ZXZZT,True,NASDAQ TEST STOCK,Q,G,False,100.0,True,N,NaN,ZXZZT,False
ZY,True,Zymergen Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZY,False
ZYME,True,Zymeworks Inc. Common Shares,N,,False,100.0,False,NaN,ZYME,ZYME,False


## Download metadata from yahoo finance

### NASDAQ symbols

In [10]:
all_tickers = traded_symbols[~traded_symbols.ETF].index.unique().to_list()
n = len(all_tickers)
print(f'# Tickers: {n:,.0f}')

# Tickers: 8,186


In [11]:
yf_tickers = yf.Tickers(all_tickers)

In [12]:
ticker_value_list = []
for ticker_key, ticker_value in yf_tickers.tickers.items():
    ticker_value_list.append(ticker_value)

In [13]:
info = []
def get(ticker_value):
    try:
        print(ticker_value)
        info.append(pd.Series(ticker_value.info).to_frame(ticker_value.ticker))
    except:
        pass

with futures.ThreadPoolExecutor(50) as executor:
    executor.map(get, ticker_value_list)

yfinance.Ticker object <A>
yfinance.Ticker object <AA>
yfinance.Ticker object <AAC>
yfinance.Ticker object <AAC.U>
yfinance.Ticker object <AAC.W>
yfinance.Ticker object <AACG>
yfinance.Ticker object <AACQ>
yfinance.Ticker object <AACQU>yfinance.Ticker object <AACQW>

yfinance.Ticker object <AAIC>
yfinance.Ticker object <AAIC$B>
yfinance.Ticker object <AAIC$C>
yfinance.Ticker object <AAL>
yfinance.Ticker object <AAMC>
yfinance.Ticker object <AAME>
yfinance.Ticker object <AAN>
yfinance.Ticker object <AAOI>
yfinance.Ticker object <AAON>
yfinance.Ticker object <AAP>
yfinance.Ticker object <AAPL>
yfinance.Ticker object <AAQC.U>
yfinance.Ticker object <AAT>
yfinance.Ticker object <AAU>
yfinance.Ticker object <AAWW>
yfinance.Ticker object <AB>
yfinance.Ticker object <ABB>
yfinance.Ticker object <ABBV>
yfinance.Ticker object <ABC>
yfinance.Ticker object <ABCB>
yfinance.Ticker object <ABCL>
yfinance.Ticker object <ABCM>
yfinance.Ticker object <ABEO>
yfinance.Ticker object <ABEV>
yfinance.Ticker

yfinance.Ticker object <AHT$F>
yfinance.Ticker object <AHT$G>
yfinance.Ticker object <AHT$H>
yfinance.Ticker object <AHT$I>
yfinance.Ticker object <AI>
yfinance.Ticker object <AIC>
yfinance.Ticker object <AIF>
yfinance.Ticker object <AIG>
yfinance.Ticker object <AIG$A>
yfinance.Ticker object <AIH>
yfinance.Ticker object <AIHS>
yfinance.Ticker object <AIKI>
yfinance.Ticker object <AIM>
yfinance.Ticker object <AIMC>
yfinance.Ticker object <AIN>
yfinance.Ticker object <AINC>
yfinance.Ticker object <AINV>
yfinance.Ticker object <AIO>
yfinance.Ticker object <AIR>
yfinance.Ticker object <AIRC>
yfinance.Ticker object <AIRG>
yfinance.Ticker object <AIRI>
yfinance.Ticker object <AIRT>
yfinance.Ticker object <AIRTP>
yfinance.Ticker object <AIRTW>
yfinance.Ticker object <AIT>
yfinance.Ticker object <AIV>
yfinance.Ticker object <AIW>
yfinance.Ticker object <AIZ>
yfinance.Ticker object <AIZN>yfinance.Ticker object <AJAX>

yfinance.Ticker object <AJAX.U>
yfinance.Ticker object <AJAX.W>
yfinance.Tick

yfinance.Ticker object <AQB>
yfinance.Ticker object <AQMS>
yfinance.Ticker object <AQN>
yfinance.Ticker object <AQNA>
yfinance.Ticker object <AQNB>
yfinance.Ticker object <AQST>
yfinance.Ticker object <AQUA>
yfinance.Ticker object <AR>
yfinance.Ticker object <ARAV>
yfinance.Ticker object <ARAY>
yfinance.Ticker object <ARBG>
yfinance.Ticker object <ARBGU>
yfinance.Ticker object <ARBGW>
yfinance.Ticker object <ARC>
yfinance.Ticker object <ARCB>
yfinance.Ticker object <ARCC>
yfinance.Ticker object <ARCE>
yfinance.Ticker object <ARCH>
yfinance.Ticker object <ARCO>
yfinance.Ticker object <ARCT>
yfinance.Ticker object <ARD>
yfinance.Ticker object <ARDC>
yfinance.Ticker object <ARDS>
yfinance.Ticker object <ARDX>
yfinance.Ticker object <ARE>
yfinance.Ticker object <AREC>
yfinance.Ticker object <ARES>
yfinance.Ticker object <ARES$A>
yfinance.Ticker object <ARGD>
yfinance.Ticker object <ARGO>
yfinance.Ticker object <ARGO$A>
yfinance.Ticker object <ARGX>
yfinance.Ticker object <ARI>
yfinance.Tic

yfinance.Ticker object <AZO>
yfinance.Ticker object <AZPN>
yfinance.Ticker object <AZRE>
yfinance.Ticker object <AZRX>
yfinance.Ticker object <AZUL>
yfinance.Ticker object <AZYO>
yfinance.Ticker object <AZZ>
yfinance.Ticker object <B>
yfinance.Ticker object <BA>
yfinance.Ticker object <BABA>
yfinance.Ticker object <BAC>
yfinance.Ticker object <BAC$B>
yfinance.Ticker object <BAC$E>
yfinance.Ticker object <BAC$K>
yfinance.Ticker object <BAC$L>
yfinance.Ticker object <BAC$M>
yfinance.Ticker object <BAC$N>
yfinance.Ticker object <BAC$O>
yfinance.Ticker object <BAC$P>
yfinance.Ticker object <BAH>
yfinance.Ticker object <BAK>
yfinance.Ticker object <BAL>
yfinance.Ticker object <BALY>
yfinance.Ticker object <BAM>
yfinance.Ticker object <BAMH>
yfinance.Ticker object <BAMI>
yfinance.Ticker object <BANC>
yfinance.Ticker object <BANC$E>
yfinance.Ticker object <BAND>
yfinance.Ticker object <BANF>
yfinance.Ticker object <BANFP>
yfinance.Ticker object <BANR>
yfinance.Ticker object <BANX>
yfinance.Ti

yfinance.Ticker object <BMTX>
yfinance.Ticker object <BMTX.W>
yfinance.Ticker object <BMY>
yfinance.Ticker object <BNED>
yfinance.Ticker object <BNFT>
yfinance.Ticker object <BNGO>
yfinance.Ticker object <BNGOW>
yfinance.Ticker object <BNKD>
yfinance.Ticker object <BNKU>
yfinance.Ticker object <BNL>
yfinance.Ticker object <BNR>
yfinance.Ticker object <BNS>
yfinance.Ticker object <BNSO>
yfinance.Ticker object <BNTC>
yfinance.Ticker object <BNTX>
yfinance.Ticker object <BNY>
yfinance.Ticker object <BOAC>
yfinance.Ticker object <BOAC.U>
yfinance.Ticker object <BOAC.W>
yfinance.Ticker object <BOAS>
yfinance.Ticker object <BOAS.U>
yfinance.Ticker object <BOAS.W>
yfinance.Ticker object <BOCH>
yfinance.Ticker object <BOE>
yfinance.Ticker object <BOH>
yfinance.Ticker object <BOKF>
yfinance.Ticker object <BOKFL>
yfinance.Ticker object <BOLT>
yfinance.Ticker object <BOMN>
yfinance.Ticker object <BOOM>
yfinance.Ticker object <BOOT>
yfinance.Ticker object <BORR>
yfinance.Ticker object <BOSC>
yfina

...

yfinance.Ticker object <SPR>
yfinance.Ticker object <SPRB>
yfinance.Ticker object <SPRO>
yfinance.Ticker object <SPRQ>
yfinance.Ticker object <SPRQ.U>
yfinance.Ticker object <SPRQ.W>
yfinance.Ticker object <SPRT>
yfinance.Ticker object <SPSC>
yfinance.Ticker object <SPT>
yfinance.Ticker object <SPTK>
yfinance.Ticker object <SPTKU>
yfinance.Ticker object <SPTKW>
yfinance.Ticker object <SPTN>
yfinance.Ticker object <SPWH>
yfinance.Ticker object <SPWR>
yfinance.Ticker object <SPXC>
yfinance.Ticker object <SPXX>
yfinance.Ticker object <SQ>
yfinance.Ticker object <SQBG>
yfinance.Ticker object <SQFT>
yfinance.Ticker object <SQM>
yfinance.Ticker object <SQNS>
yfinance.Ticker object <SQZ>
yfinance.Ticker object <SR>
yfinance.Ticker object <SR$A>
yfinance.Ticker object <SRAC>
yfinance.Ticker object <SRACU>
yfinance.Ticker object <SRACW>
yfinance.Ticker object <SRAX>
yfinance.Ticker object <SRC>
yfinance.Ticker object <SRC$A>
yfinance.Ticker object <SRCE>
yfinance.Ticker object <SRCL>
yfinance.T

yfinance.Ticker object <TCFCP>
yfinance.Ticker object <TCI>
yfinance.Ticker object <TCMD>
yfinance.Ticker object <TCOM>
yfinance.Ticker object <TCON>
yfinance.Ticker object <TCPC>
yfinance.Ticker object <TCRR>
yfinance.Ticker object <TCS>
yfinance.Ticker object <TCVA>
yfinance.Ticker object <TCX>
yfinance.Ticker object <TD>
yfinance.Ticker object <TDA>
yfinance.Ticker object <TDAC>
yfinance.Ticker object <TDACU>
yfinance.Ticker object <TDACW>
yfinance.Ticker object <TDC>
yfinance.Ticker object <TDE>
yfinance.Ticker object <TDF>
yfinance.Ticker object <TDG>
yfinance.Ticker object <TDI>
yfinance.Ticker object <TDJ>
yfinance.Ticker object <TDOC>
yfinance.Ticker object <TDS>
yfinance.Ticker object <TDS$U>yfinance.Ticker object <TDUP>yfinance.Ticker object <TDW>
yfinance.Ticker object <TDW.A>


yfinance.Ticker object <TDW.B>
yfinance.Ticker object <TDW.W>
yfinance.Ticker object <TDY>
yfinance.Ticker object <TEAF>
yfinance.Ticker object <TEAM>
yfinance.Ticker object <TECH>
yfinance.Ticker ob

yfinance.Ticker object <TSRI>
yfinance.Ticker object <TT>
yfinance.Ticker object <TTC>
yfinance.Ticker object <TTCF>
yfinance.Ticker object <TTD>
yfinance.Ticker object <TTEC>
yfinance.Ticker object <TTEK>
yfinance.Ticker object <TTGT>
yfinance.Ticker object <TTI>
yfinance.Ticker object <TTM>
yfinance.Ticker object <TTMI>
yfinance.Ticker object <TTNP>
yfinance.Ticker object <TTOO>
yfinance.Ticker object <TTP>
yfinance.Ticker object <TTWO>
yfinance.Ticker object <TU>
yfinance.Ticker object <TUFN>
yfinance.Ticker object <TUGCU>
yfinance.Ticker object <TUP>
yfinance.Ticker object <TURN>
yfinance.Ticker object <TUSK>
yfinance.Ticker object <TUYA>
yfinance.Ticker object <TV>
yfinance.Ticker object <TVAC>
yfinance.Ticker object <TVACU>
yfinance.Ticker object <TVACW>
yfinance.Ticker object <TVC>
yfinance.Ticker object <TVE>
yfinance.Ticker object <TVTX>
yfinance.Ticker object <TVTY>
yfinance.Ticker object <TW>
yfinance.Ticker object <TWCT>
yfinance.Ticker object <TWCTU>
yfinance.Ticker object

yfinance.Ticker object <VHAQ.U>
yfinance.Ticker object <VHAQ.W>
yfinance.Ticker object <VHC>
yfinance.Ticker object <VHI>
yfinance.Ticker object <VIAC>
yfinance.Ticker object <VIACA>
yfinance.Ticker object <VIACP>
yfinance.Ticker object <VIAO>
yfinance.Ticker object <VIAV>
yfinance.Ticker object <VICI>
yfinance.Ticker object <VICR>
yfinance.Ticker object <VIEW>
yfinance.Ticker object <VIEWW>
yfinance.Ticker object <VIH>
yfinance.Ticker object <VIHAU>
yfinance.Ticker object <VIHAW>
yfinance.Ticker object <VII>
yfinance.Ticker object <VIIAU>
yfinance.Ticker object <VIIAW>
yfinance.Ticker object <VINC>
yfinance.Ticker object <VINO>
yfinance.Ticker object <VINP>
yfinance.Ticker object <VIOT>
yfinance.Ticker object <VIPS>
yfinance.Ticker object <VIR>
yfinance.Ticker object <VIRC>
yfinance.Ticker object <VIRI>
yfinance.Ticker object <VIRT>
yfinance.Ticker object <VIRX>
yfinance.Ticker object <VISL>
yfinance.Ticker object <VIST>
yfinance.Ticker object <VITL>
yfinance.Ticker object <VIV>
yfina

yfinance.Ticker object <WPG$H>
yfinance.Ticker object <WPG$I>
yfinance.Ticker object <WPM>
yfinance.Ticker object <WPP>
yfinance.Ticker object <WPRT>
yfinance.Ticker object <WRAP>
yfinance.Ticker object <WRB>
yfinance.Ticker object <WRB$D>
yfinance.Ticker object <WRB$E>
yfinance.Ticker object <WRB$F>
yfinance.Ticker object <WRB$G>
yfinance.Ticker object <WRB$H>
yfinance.Ticker object <WRE>
yfinance.Ticker object <WRI>
yfinance.Ticker object <WRK>
yfinance.Ticker object <WRLD>
yfinance.Ticker object <WRN>
yfinance.Ticker object <WSBC>
yfinance.Ticker object <WSBCP>
yfinance.Ticker object <WSBF>
yfinance.Ticker object <WSC>
yfinance.Ticker object <WSFS>
yfinance.Ticker object <WSM>
yfinance.Ticker object <WSO>
yfinance.Ticker object <WSO.B>
yfinance.Ticker object <WSR>
yfinance.Ticker object <WST>
yfinance.Ticker object <WSTG>
yfinance.Ticker object <WTBA>
yfinance.Ticker object <WTER>
yfinance.Ticker object <WTFC>
yfinance.Ticker object <WTFCM>
yfinance.Ticker object <WTFCP>
yfinance.Ti

In [15]:
info = pd.concat(info, axis=1).dropna(how='all').T
info = info.apply(pd.to_numeric, errors='ignore')
info.to_hdf(results_path / 'data.h5', 'stocks/info')

In [16]:
info

,logo_url,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,country,companyOfficers,website,...,pegRatio,shortPercentOfFloat,sharesShortPriorMonth,impliedSharesOutstanding,category,fiveYearAverageReturn,regularMarketPrice,state,address2,err
AAIC,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAME,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABB,https://logo.clearbit.com/global.abb,8050,Industrials,105330.0,ABB Ltd manufactures and sells electrification...,Zurich,41 43 317 71 11,Switzerland,NaN,http://www.global.abb/group/,...,1.84,NaN,4495287.0,NaN,None,NaN,34.110,NaN,NaN,NaN
ABST,https://logo.clearbit.com/absolute.com,V7X 1K8,Technology,526.0,"Absolute Software Corporation develops, market...",Vancouver,604-730-9851,Canada,NaN,http://www.absolute.com,...,NaN,NaN,767596.0,NaN,None,NaN,14.340,BC,Suite 1400 1055 Dunsmuir Street PO Box 49211,NaN
AACQ,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.985,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXIET,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZWZZT,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZXYZ.A,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZXZZT,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Download adjusted price data using yfinance

In [18]:
prices_adj = []
start = time()
for i, chunk in enumerate(chunks(all_tickers, 100), 1):
    prices_adj.append(yf.download(chunk, period='max', auto_adjust=True).stack(-1))

    per_ticker = (time()-start) / (i * 100)
    to_do = n - (i * 100)
    to_go = to_do * per_ticker    
    print(f'Success: {len(prices_adj):5,.0f}/{i:5,.0f} | To go: {format_time(to_go)} ({to_do:5,.0f})')

[*********************100%***********************]  100 of 100 completed

20 Failed downloads:
- ACND.U: No data found, symbol may be delisted
- ABR$A: No data found, symbol may be delisted
- ABR$C: No data found, symbol may be delisted
- ACEVW: 1d data not available for startTime=-2208988800 and endTime=1620572238. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ACII.W: No data found, symbol may be delisted
- AAIC$C: No data found, symbol may be delisted
- ACKIW: 1d data not available for startTime=-2208988800 and endTime=1620572239. Only 100 years worth of day granularity data are allowed to be fetched per request.
- AACQW: 1d data not available for startTime=-2208988800 and endTime=1620572239. Only 100 years worth of day granularity data are allowed to be fetched per request.
- AAQC.U: No data found, symbol may be delisted
- ACIC.W: No data found, symbol may be delisted
- ACQR: 1d data not available for startTime=-2208988800 and endTime=16205722

Success:     5/    5 | To go: 00:14:20 (7,686)
[*********************100%***********************]  100 of 100 completed

17 Failed downloads:
- ARKOW: 1d data not available for startTime=-2208988800 and endTime=1620572294. Only 100 years worth of day granularity data are allowed to be fetched per request.
- APOPW: 1d data not available for startTime=-2208988800 and endTime=1620572294. Only 100 years worth of day granularity data are allowed to be fetched per request.
- APXTW: 1d data not available for startTime=-2208988800 and endTime=1620572294. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ARES$A: No data found, symbol may be delisted
- APO$B: No data found, symbol may be delisted
- ARR$C: No data found, symbol may be delisted
- APGB.U: No data found, symbol may be delisted
- APSG.W: No data found, symbol may be delisted
- APO$A: No data found, symbol may be delisted
- ARBGW: 1d data not available for startTime=-2208988800 and endTime=162057229

Success:    10/   10 | To go: 00:13:05 (7,186)
[*********************100%***********************]  100 of 100 completed

17 Failed downloads:
- BIOTW: 1d data not available for startTime=-2208988800 and endTime=1620572347. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BML$G: No data found, symbol may be delisted
- BIP$B: No data found, symbol may be delisted
- BITE.W: No data found, symbol may be delisted
- BITE.U: No data found, symbol may be delisted
- BLUA.W: No data found, symbol may be delisted
- BIO.B: 1d data not available for startTime=-2208988800 and endTime=1620572351. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BHSEW: 1d data not available for startTime=-2208988800 and endTime=1620572352. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BML$L: No data found, symbol may be delisted
- BML$H: No data found, symbol may be delisted
- BIP$A: No data found, symbol 

Success:    16/   16 | To go: 00:12:01 (6,586)
[*********************100%***********************]  100 of 100 completed

18 Failed downloads:
- CLNY$J: No data found, symbol may be delisted
- CLRMW: 1d data not available for startTime=-2208988800 and endTime=1620572413. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CMRE$D: No data found, symbol may be delisted
- CMS$B: No data found, symbol may be delisted
- CLIM.W: No data found, symbol may be delisted
- CLNY$H: No data found, symbol may be delisted
- CLNNW: 1d data not available for startTime=-2208988800 and endTime=1620572416. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CMRE$C: No data found, symbol may be delisted
- CMRE$B: No data found, symbol may be delisted
- CMO$E: No data found, symbol may be delisted
- CLIM.U: No data found, symbol may be delisted
- CLNY$G: No data found, symbol may be delisted
- CMIIW: 1d data not available for startTime=-2208

Success:    22/   22 | To go: 00:11:01 (5,986)
[*********************100%***********************]  100 of 100 completed

20 Failed downloads:
- DSKEW: 1d data not available for startTime=-2208988800 and endTime=1620572481. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DS$C: No data found, symbol may be delisted
- DSX$B: No data found, symbol may be delisted
- DMYQ.W: No data found, symbol may be delisted
- DWIN.W: No data found, symbol may be delisted
- DS$D: No data found, symbol may be delisted
- DMYQ.U: No data found, symbol may be delisted
- DNMR.W: No data found, symbol may be delisted
- DTOCW: 1d data not available for startTime=-2208988800 and endTime=1620572483. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DUK$A: No data found, symbol may be delisted
- DTLA$: No data found, symbol may be delisted
- DMYI.W: No data found, symbol may be delisted
- DNZ.U: No data found, symbol may be delisted
- DWIN.U

...

Success:    53/   53 | To go: 00:05:16 (2,886)
[*********************100%***********************]  100 of 100 completed

24 Failed downloads:
- NSTD.U: No data found, symbol may be delisted
- NRZ$A: No data found, symbol may be delisted
- NRZ$C: No data found, symbol may be delisted
- NTEST.A: No data found, symbol may be delisted
- NS$B: No data found, symbol may be delisted
- NSH.W: No data found, symbol may be delisted
- NVVEW: 1d data not available for startTime=-2208988800 and endTime=1620572821. Only 100 years worth of day granularity data are allowed to be fetched per request.
- NTEST: 1d data not available for startTime=-2208988800 and endTime=1620572821. Only 100 years worth of day granularity data are allowed to be fetched per request.
- NSPR.B: No data found, symbol may be delisted
- NS$A: No data found, symbol may be delisted
- NTEST.B: No data found, symbol may be delisted
- NUVB.W: No data found, symbol may be delisted
- NSTD.W: No data found, symbol may be delisted
- NSA

Success:    60/   60 | To go: 00:04:03 (2,186)
[*********************100%***********************]  100 of 100 completed

29 Failed downloads:
- PUK$: No data found, symbol may be delisted
- PTIXW: 1d data not available for startTime=-2208988800 and endTime=1620572905. Only 100 years worth of day granularity data are allowed to be fetched per request.
- PSA$O: No data found, symbol may be delisted
- PSA$D: No data found, symbol may be delisted
- PSA$L: No data found, symbol may be delisted
- PSA$F: No data found, symbol may be delisted
- PTK.W: No data found, symbol may be delisted
- PSB$X: No data found, symbol may be delisted
- PSA$H: No data found, symbol may be delisted
- PSA$G: No data found, symbol may be delisted
- PSB$Z: No data found, symbol may be delisted
- PSB$Y: No data found, symbol may be delisted
- PSFE.W: No data found, symbol may be delisted
- PSTH.W: No data found, symbol may be delisted
- PSA$C: No data found, symbol may be delisted
- PUK$A: No data found, symbol may

Success:    66/   66 | To go: 00:02:55 (1,586)
[*********************100%***********************]  100 of 100 completed

13 Failed downloads:
- SGLBW: 1d data not available for startTime=-2208988800 and endTime=1620572968. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SFB: 1d data not available for startTime=-2208988800 and endTime=1620572968. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHO$E: No data found, symbol may be delisted
- SFTW.W: No data found, symbol may be delisted
- SGAMW: 1d data not available for startTime=-2208988800 and endTime=1620572969. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SF$B: No data found, symbol may be delisted
- SHIPW: 1d data not available for startTime=-2208988800 and endTime=1620572970. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHIPZ: 1d data not available for startTime=-2208988800 an

[*********************100%***********************]  100 of 100 completed

21 Failed downloads:
- THMAW: 1d data not available for startTime=-2208988800 and endTime=1620573035. Only 100 years worth of day granularity data are allowed to be fetched per request.
- TINV.W: No data found, symbol may be delisted
- TLGA.U: No data found, symbol may be delisted
- TLMDW: 1d data not available for startTime=-2208988800 and endTime=1620573035. Only 100 years worth of day granularity data are allowed to be fetched per request.
- TFC$R: No data found, symbol may be delisted
- TFC$H: No data found, symbol may be delisted
- TFC$I: No data found, symbol may be delisted
- TFC$O: No data found, symbol may be delisted
- TMAC.U: No data found, symbol may be delisted
- TGH$A: No data found, symbol may be delisted
- TMAC.W: No data found, symbol may be delisted
- THWWW: 1d data not available for startTime=-2208988800 and endTime=1620573038. Only 100 years worth of day granularity data are allowed to be fetc

Success:    78/   78 | To go: 00:00:43 (  386)
[*********************100%***********************]  100 of 100 completed

10 Failed downloads:
- WBS$F: No data found, symbol may be delisted
- VYGG.W: No data found, symbol may be delisted
- VYGG.U: No data found, symbol may be delisted
- VTAQW: 1d data not available for startTime=-2208988800 and endTime=1620573104. Only 100 years worth of day granularity data are allowed to be fetched per request.
- WCC$A: No data found, symbol may be delisted
- WARR.U: No data found, symbol may be delisted
- VST.A: No data found, symbol may be delisted
- VTAQR: 1d data not available for startTime=-2208988800 and endTime=1620573107. Only 100 years worth of day granularity data are allowed to be fetched per request.
- WARR.W: No data found, symbol may be delisted
- VRMEW: 1d data not available for startTime=-2208988800 and endTime=1620573109. Only 100 years worth of day granularity data are allowed to be fetched per request.
Success:    79/   79 | To go: 

In [19]:
prices_adj = (pd.concat(prices_adj)
              .dropna(how='all', axis=1)
              .rename(columns=str.lower)
              .swaplevel())

In [20]:
prices_adj.index.names = ['ticker', 'date']

In [21]:
len(prices_adj.index.unique('ticker'))

6842

### Remove outliers

In [22]:
df = prices_adj.close.unstack('ticker')
pmax = df.pct_change().max()
pmin = df.pct_change().min()
to_drop = pmax[pmax > 1].index.union(pmin[pmin<-1].index)
len(to_drop)

794

In [23]:
prices_adj = prices_adj.drop(to_drop, level='ticker')

In [24]:
len(prices_adj.index.unique('ticker'))

6048

In [25]:
prices_adj.sort_index().loc[idx[:, '1990': '2019'], :].to_hdf(results_path / 'data.h5', 
                                                              'stocks/prices/adjusted')